## Charactor protrait extractor

**Current Method**  
1. 從每次的 advice 提取出 summary            
2. 迭代方式暫時使用疊加

---

**TODO**
1. protrait extractor 輸入加上前一次的 protrait
2. 寫成 tool 讓 self_play 使用

In [28]:
from typing import Dict
from openai import OpenAI
from dotenv import load_dotenv
import os
load_dotenv()

Memory = Dict[str, list]

In [11]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [13]:
SYSTEM_PROMPT = """
You are a memory extractor that analyzes character feedback and organizes it into specific categories. Based on the given feedback, identify and extract insights into the following five areas, using clear and concise natural language:

1. traits: The character's personality and defining attributes. Examples include being defensive, proud, vulnerable, possessive, empathetic, assertive, manipulative, shy, cunning, or loyal.

2. tone: Style and tone of the character's language. Examples include sarcastic, casual, formal, passive-aggressive, confrontational, cold, enthusiastic, overly emotional, or deadpan.

3. inner_conflict: Any signs of emotional struggle or layered feelings. Examples include hiding fear behind confidence, being torn between duty and desire, wanting help but refusing it out of pride, or expressing affection while pretending to be indifferent.

4. interaction_dynamics: Relationship and tension with the user or other characters. Examples include rivalry, friendship, power imbalance, mistrust, dependency, manipulation, mutual respect, or emotional detachment.

5. improvement_suggestions: Concrete recommendations for improving the next generation. Examples include enhancing emotional contrast, making tone more consistent, increasing character distinctiveness, showing more tension in dialogue, or clarifying motivations.

Return the result in the following JSON format:
{
  "traits": [...],
  "tone": [...],
  "inner_conflict": [...],
  "interaction_dynamics": [...],
  "improvement_suggestions": [...]
}
"""

In [14]:
def extract_memory_with_llm(discriminator_advice: str) -> Memory:
    """
    Uses an LLM to extract memory components from discriminator feedback
    """
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": f"Here is the feedback:\n{discriminator_advice}"}
        ],
        temperature=0.3
    )
    content = response.choices[0].message.content
    try:
        memory_dict = eval(content)
        return memory_dict
    except Exception as e:
        print("Failed to parse response format:", e)
        return {}

In [15]:
def merge_memory(old: Memory, new: Memory) -> Memory:
    return {
        k: list(set(old.get(k, []) + new.get(k, []))) for k in old.keys()
    }

In [18]:
def update_memory(prev_memory: Memory, discriminator_advice: str) -> Memory:
    new_memory = extract_memory_with_llm(discriminator_advice)
    return merge_memory(prev_memory, new_memory)

#### v0-0

In [20]:
last_memory = {
    "traits": [],
    "tone": [],
    "inner_conflict": [],
    "interaction_dynamics": [],
    "improvement_suggestions": []
}

In [21]:
discriminator_advice = """**Feedback on Assistant's Original Response:**\n\n1. **Tone and Language**: The original response captures a casual, somewhat arrogant tone that fits Marco's character well. The use of phrases like \"had a bit too much fun\" and \"Spoiler alert: they couldn't!\" aligns with a colloquial style, making it feel more like a real conversation. However, the tone could be slightly more antagonistic or dismissive to better reflect the rivalry aspect of Marco's character. For example, he might want to assert his superiority more aggressively.\n\n2. **Character Consistency**: The response does a good job of portraying Marco's drunken state while maintaining his characteristic bravado. However, given the background of animosity, Marco might express more irritation or disdain towards the user, even in a vulnerable state. Adding a line that belittles the user's concern or mocks them for worrying could enhance the rivalry dynamic.\n\n3. **Emotional Depth**: While the response shows Marco's cockiness, it could benefit from a hint of vulnerability or defensiveness that comes from being in a compromised situation. Perhaps he could briefly acknowledge that he doesn't usually get this way, which could add complexity to his character, showing that beneath the bravado, he may be struggling with his own image.\n\n4. **Interaction with User**: The original reply lacks a direct response to the user's concern, which is vital in maintaining the conversational flow. Marco could acknowledge the user's question but twist it to sound dismissive, reinforcing his antagonistic nature. For example, he might say something like, \"Why do you care? It's not like you can help me anyway.\"\n\n**Revised Suggestion**: \n\"Yeah, I’m fine. Just had a bit too much fun. Some idiots at the party thought they could out-drink me. Spoiler alert: they couldn't! But seriously, why do you care? It’s not like you can help me anyway. Just keep your mouth shut about this, alright? I can’t have you ruining my reputation.\" *He leans against the wall, a smirk on his face, but his eyes betray a hint of insecurity.* \n\nThis revision keeps the original elements while enhancing the rivalry and dismissive nature of Marco's character."""

In [22]:
updated = update_memory(last_memory, discriminator_advice)
for k, v in updated.items():
    print(f"{k}: {v}\n")

traits: ['arrogant', 'antagonistic', 'defensive', 'bravado', 'vulnerable']

tone: ['colloquial', 'dismissive', 'casual', 'irritated']

inner_conflict: ['struggling with self-image', 'hiding insecurity behind arrogance']

interaction_dynamics: ['mocking', 'rivalry', 'dismissive towards user']

improvement_suggestions: ['add belittling remarks', 'show more emotional depth', 'enhance antagonistic tone', 'acknowledge user concern with dismissal']



#### v0-1

In [24]:
last_memory = updated

In [25]:
discriminator_advice = """**Feedback:**\n\nThe original response from the assistant does a good job of capturing Marco's character as an antagonistic and possessive rival. Here are some specific points of feedback:\n\n1. **Colloquial Language**: The response uses a casual tone that fits well with an online chat experience. Phrases like \"Ugh, I’m fine\" and \"some idiots thought they could out-drink me\" reflect a conversational style that is appropriate for Marco's character.\n\n2. **Antagonistic Attitude**: Marco's dismissive attitude towards the user (\"why do you care?\") and his concern about reputation are consistent with his possessive and protective traits. This aligns well with the background provided.\n\n3. **Confidence with Vulnerability**: The smirk and the attempt to play it cool despite being visibly intoxicated show a good balance of bravado and vulnerability, which adds depth to Marco's character.\n\n4. **Interaction Dynamics**: The way Marco interacts with the user—trying to downplay his situation while also seeking a level of understanding—captures the complicated nature of their relationship. His request not to tell anyone is a nice touch that emphasizes his rivalry and need for control over his image.\n\nOverall, the response fits well within the established background of Marco's character. It effectively portrays his personality and the dynamics of his relationship with the user. Keep up the good work in maintaining that balance of rivalry and complexity in future interactions!"""

In [26]:
updated = update_memory(last_memory, discriminator_advice)
for k, v in updated.items():
    print(f"{k}: {v}\n")

traits: ['possessive', 'arrogant', 'antagonistic', 'confident', 'defensive', 'protective', 'bravado', 'vulnerable']

tone: ['irritated', 'colloquial', 'dismissive', 'conversational', 'casual']

inner_conflict: ['struggling with self-image', 'hiding insecurity behind arrogance', 'hiding insecurity behind bravado', 'confidence with vulnerability']

interaction_dynamics: ['need for control', 'rivalry', 'mocking', 'dismissive towards user', 'seeking understanding']

improvement_suggestions: ['maintain balance of rivalry and complexity', 'enhance emotional depth in interactions', 'acknowledge user concern with dismissal', 'enhance antagonistic tone', 'add belittling remarks', 'show more emotional depth']

